In [2]:
from process.dataset import prepare_dataset

In [3]:
path_data="./pycharm_issues.json"

## Prepare train data

In [ ]:
train_data, clear_data, topic_model =prepare_dataset(path_data)

Batches:   0%|          | 0/111 [00:00<?, ?it/s]

/Users/Olga.Lavrichenko/opt/anaconda3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


## Learn model

In [ ]:
from tp.model_class import ModelCTM

In [ ]:
model_tp=ModelCTM(topic_model, 768, 10, 5)

In [ ]:
model_tp.fit(train_data)

Epoch: [1/70]	 Seen Samples: [22014/1540980]	Train Loss: 67.89619912284157	Time: 0:00:08.311317: : 1it [00:08,  8.31s/it]